In [1]:
#Num of candidates
import random
import pandas as pd
import math
import operator
import numpy as np



In [2]:
#Takes as input a dataframe with two columns: a 'ballot' column
#and a 'Count' column. The Count column gives the number of ballots of
#each type that were cast. Returns the IRV winner(s).
def IRV(frame):
    num_cands_remaining=len(frame.iloc[0]['ballot'])
    n=num_cands_remaining
    cand_dict={ }
    keys=range(n)
    values=[]
    
    for k in range(len(frame.iloc[0]['ballot'])):
        values.append(frame.iloc[0]['ballot'][k])
    
    for i in keys:
        cand_dict[i]=values[i]
    while num_cands_remaining>1:
        scores=[0 for i in range(n)]
        for k in range(len(frame)):
            for l in range(n):
                if frame.iloc[k]['ballot'][0]==cand_dict[l]:
                    scores[l]+=frame.iloc[k]['Count']
        for k in range(len(scores)):
            if scores[k]>sum(frame['Count'])/2:
                return [str(cand_dict[k])]

        min_score=min(i for i in scores if i>0)
        positions_of_lowest_score=[i for i,v in enumerate(scores) if v==min_score]
        #print(positions_of_lowest_score)
        
        #Tie at the top, record the multiple winners
        if num_cands_remaining==len(positions_of_lowest_score):
            return [cand_dict[i] for i in positions_of_lowest_score]
            break
        num_cands_remaining-=len(positions_of_lowest_score)

        for k in range(len(frame)):
            for l in positions_of_lowest_score:
                frame.at[k,'ballot']=frame.at[k,'ballot'].replace(cand_dict[l],'')
        #print(data)
        frame=pd.DataFrame(frame.groupby(['ballot'],as_index=False)['Count'].sum())
        #print(data)

    IRV_winner=frame['ballot'][0]
    return [IRV_winner]

#Takes as input a dataframe with an election. Returns the CL if there is one,
#otherwise returns False
def Cond_loser(frame):
    num_cands_remaining=len(frame.iloc[0]['ballot'])
    n=num_cands_remaining
    cands_dict={ }
    
    keys=range(n)
    values=[]
    
    for k in range(len(frame.iloc[0]['ballot'])):
        values.append(frame.iloc[0]['ballot'][k])
    
    for i in keys:
        cands_dict[i]=values[i]
    Cond_scores=[0 for i in range(len(cands_dict))]
    #Get all pairings
    possible_pairings=[]
    for i in range(len(cands_dict)):
        for j in range(len(cands_dict)):
            if i<j:
                possible_pairings.append([i,j])

    for pairing in possible_pairings:
        #print(pairing)
        pairing_scores=[0,0]
        Cand1=pairing[0]
        Cand2=pairing[1]
        for k in range(len(frame)):
            for l in range(len(cands_dict)):
                if frame.iloc[k]['ballot'][l]==cands_dict[Cand1]:
                    pairing_scores[0]+=frame.iloc[k]['Count']
                    #print(Cand1, cands.index(Cand1),pairing_scores)
                    break
                if frame.iloc[k]['ballot'][l]==cands_dict[Cand2]:
                    pairing_scores[1]+=frame.iloc[k]['Count']
                    #print(Cand2, cands.index(Cand2),pairing_scores)
                    break
        #print(pairing_scores)
        if pairing_scores[0]>pairing_scores[1]:
            Cond_scores[Cand1]+=1
        if pairing_scores[0]<pairing_scores[1]:
            Cond_scores[Cand2]+=1
        
    
    if 0 not in Cond_scores:
        return False
    else:
        return cands_dict[Cond_scores.index(0)]
    
   

#Takes as input a dataframe with two columns: a 'ballot' column
#and a 'Count' column. The Count column gives the number of ballots of
#each type that were cast. Returns the plurality winner(s).
def plurality(frame):
    n=len(frame.iloc[0]['ballot'])
    cand_dict={ }
    keys=range(n)
    values=[]
    
    for k in range(len(frame.iloc[0]['ballot'])):
        values.append(frame.iloc[0]['ballot'][k])
    
    for i in keys:
        cand_dict[i]=values[i]
    
    scores=[0 for i in range(n)]
    for k in range(len(frame)):
        for l in range(n):
            if frame.iloc[k]['ballot'][0]==cand_dict[l]:
                scores[l]+=frame.iloc[k]['Count']
    
    max_score=max(scores)
    positions_of_highest_score=[i for i,v in enumerate(scores) if v==max_score]
    winning_cands=[cand_dict[i] for i in positions_of_highest_score]
    return winning_cands

def powerset(seq):
    """
    Returns all the subsets of this set. This is a generator.
    """
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

            #Checks if all of the elements of a list are distinct.
def all_different(mylist):
    for i in range(len(mylist)):
        for j in range(len(mylist)):
            if i!=j:
                if mylist[i]==mylist[j]:
                    return False
    return True

#Given a ranking of candidates, checks if the ranking is single-peaked
def is_SP_ranking(ranking):
    top_ranking=ranking[0]
    left_rankings=[i for i in range(top_ranking)]
    right_rankings=[]
    for k in range(len(ranking)-len(left_rankings)-1):
        right_rankings.append(ranking[0]+1+k)
    for i in left_rankings:
        for j in left_rankings:
            if i<j and ranking.index(i)<ranking.index(j):
                return False
    
    for i in right_rankings:
        for j in right_rankings:
            if i<j and ranking.index(i)>ranking.index(j):
                return False
    return True
    


def clones(frame,l):
    cand_set=list(frame.iloc[0]['ballot'])
    cand_set.remove(str(l))
    clone_set=[]
    n=len(frame.iloc[0]['ballot'])
    
    for cand in cand_set:
        clone=True
        
        for k in range(len(frame)):
            if abs(frame.iloc[k]['ballot'].index(cand)-frame.iloc[k]['ballot'].index(str(l)))>1:
                clone=False
                break
        if clone:
            clone_set.append(cand)
    return clone_set

def almost_clones(frame, l, p):
    cand_set=list(frame.iloc[0]['ballot'])
    cand_set.remove(str(l))
    clone_set=[]
    n=len(frame.iloc[0]['ballot'])
    for cand in cand_set:
        clone=True
        count=0
        for k in range(len(frame)):
            if abs(frame.iloc[k]['ballot'].index(cand)-frame.iloc[k]['ballot'].index(str(l)))==1:
                count+=frame.iloc[k]['Count']
        if count/sum(frame['Count'])>p:
            clone_set.append(cand)
    return clone_set

In [14]:
#impartial culture. 
#n is number of candidates, electorate_size is the number of voters.
n=3
electorate_size=1001
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))



column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

count=0
count1=0
count2=0

for a in range(10000):
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    if data.at[0,'Count']+data.at[1,'Count']<electorate_size/2:
        count1+=1
    
    if data.at[0,'Count']+data.at[1,'Count']<electorate_size/2 and data.at[0,'Count']+data.at[1,'Count']>data.at[2,'Count']+data.at[3,'Count'] and data.at[2,'Count']+data.at[3,'Count']>data.at[4,'Count']+data.at[5,'Count']:
        count+=1

print(count1)
print(count2)
print(count)
    
    
    

10000
0
1599


In [11]:
#impartial culture. 
#n is number of candidates, electorate_size is the number of voters.
n=3
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))



column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

count=0
count1=0
count2=0

for a in range(10000):
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    data_copy=data.copy(deep=True) 
    
    plurality_winners=plurality(data_copy)
   
    A_plurality_winner=False
    if plurality_winners==['0']:
        A_plurality_winner=True
        count1+=1
    when_B_drops_C_wins=False
    if data.at[0,'Count']+data.at[1,'Count']+data.at[2,'Count']<data.at[3,'Count']+data.at[4,'Count']+data.at[5,'Count']:
        when_B_drops_C_wins=True
        count2+=1
    when_C_drops_A_wins=False
    if data.at[0,'Count']+data.at[1,'Count']+data.at[4,'Count']>data.at[3,'Count']+data.at[2,'Count']+data.at[5,'Count']:
        when_C_drops_A_wins=True
    if A_plurality_winner and when_B_drops_C_wins and when_C_drops_A_wins:
        count+=1

print(count1)
print(count2)
print(count)
    
    
    

3249
4862
376


In [15]:
#IAC simulations for CL
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap

def soc_inac(ranksmaps, rankmapcount, ncand):
    soc_inac_ensemble = []
    condi_position = pd.DataFrame(
        ranksmaps, columns=[i for i in range(1, ncand+1)])
    for i in range(1, ncand+1):
        sum_nsup = 0
        sum_ninf = 0
        nsup = condi_position[condi_position[i] <= int(ncand/2)].index.values
        ninf = condi_position[condi_position[i] > np.ceil(ncand/2)].index.values
        for j in range(len(nsup)):
            sum_nsup = sum_nsup + rankmapcount[nsup[j]]
        for j in range(len(ninf)):
            sum_ninf = sum_ninf + rankmapcount[ninf[j]]
        if sum_ninf > sum_nsup:
            soc_inac_ensemble.append(str(i))

    return soc_inac_ensemble
def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=601
model='IAC'
possible_ballots=[]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=100000
count=0
while a>0:
    
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
    
        CL=Cond_loser(data)

        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            #print("CL is a plurality winner ",CL)
            #print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                #print("CL is a plurality spoiler ",CL)
                #print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            #print("CL is an IRV winner ",CL)
            #print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                #print("CL is an IRV spoiler ",CL)
                #print(data)
                CL_IRV_spoiler+=1
        a-=1

print("Model= ",model, " n=",n, " V=",electorate_size)           
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)



Model=  IAC  n= 3  V= 601
Number of elections in which CL is a plurality spoiler  15022
Number of elections in which CL is a plurality winner  5169
Number of elections in which CL is an IRV spoiler  5363
Number of elections in which CL is an IRV winner  29


In [16]:
#IAC simulations for CL
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap

def soc_inac(ranksmaps, rankmapcount, ncand):
    soc_inac_ensemble = []
    condi_position = pd.DataFrame(
        ranksmaps, columns=[i for i in range(1, ncand+1)])
    for i in range(1, ncand+1):
        sum_nsup = 0
        sum_ninf = 0
        nsup = condi_position[condi_position[i] <= int(ncand/2)].index.values
        ninf = condi_position[condi_position[i] > np.ceil(ncand/2)].index.values
        for j in range(len(nsup)):
            sum_nsup = sum_nsup + rankmapcount[nsup[j]]
        for j in range(len(ninf)):
            sum_ninf = sum_ninf + rankmapcount[ninf[j]]
        if sum_ninf > sum_nsup:
            soc_inac_ensemble.append(str(i))

    return soc_inac_ensemble
def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=600
model='IAC'
possible_ballots=[]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=100000
count=0
while a>0:
    
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
    
        CL=Cond_loser(data)

        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            #print("CL is a plurality winner ",CL)
            #print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                #print("CL is a plurality spoiler ",CL)
                #print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            #print("CL is an IRV winner ",CL)
            #print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                #print("CL is an IRV spoiler ",CL)
                #print(data)
                CL_IRV_spoiler+=1
        a-=1

print("Model= ",model, " n=",n, " V=",electorate_size)     
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)




Model=  IAC  n= 3  V= 600
Number of elections in which CL is a plurality spoiler  15212
Number of elections in which CL is a plurality winner  5055
Number of elections in which CL is an IRV spoiler  5340
Number of elections in which CL is an IRV winner  312


In [17]:
#IAC simulations
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap

def soc_inac(ranksmaps, rankmapcount, ncand):
    soc_inac_ensemble = []
    condi_position = pd.DataFrame(
        ranksmaps, columns=[i for i in range(1, ncand+1)])
    for i in range(1, ncand+1):
        sum_nsup = 0
        sum_ninf = 0
        nsup = condi_position[condi_position[i] <= int(ncand/2)].index.values
        ninf = condi_position[condi_position[i] > np.ceil(ncand/2)].index.values
        for j in range(len(nsup)):
            sum_nsup = sum_nsup + rankmapcount[nsup[j]]
        for j in range(len(ninf)):
            sum_ninf = sum_ninf + rankmapcount[ninf[j]]
        if sum_ninf > sum_nsup:
            soc_inac_ensemble.append(str(i))

    return soc_inac_ensemble
def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=501
possible_ballots=[]
candmap = gen_cand_map(n)
election=gen_votemap_iac(electorate_size,candmap)

plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l+1): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)

Num elections with a plurality spoiler cand:  20309
Num elections with a plurality spoiler set:  20309
Num elections with an IRV spoiler cand:  9976
Num elections with an IRV spoiler set:  9976
Num elections with a plurality clone spoiler  4
Num elections with a plurality clone spoiler who spoiled the election for their clone 4
Num elections with a plurality almost clone spoiler  1169
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 1156
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  102
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 102


In [23]:
#IAC simulations FOR REAL
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=601
possible_ballots=[]
candmap = gen_cand_map(n)
election=gen_votemap_iac(electorate_size,candmap)

plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l+1): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(1,n+1)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(1,n+1):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(1,n+1):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)

Num elections with a plurality spoiler cand:  40242
Num elections with a plurality spoiler set:  40242
Num elections with an IRV spoiler cand:  21187
Num elections with an IRV spoiler set:  21187
Num elections with a plurality clone spoiler  6
Num elections with a plurality clone spoiler who spoiled the election for their clone 6
Num elections with a plurality almost clone spoiler  1931
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 1893
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  208
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 208


In [289]:
#impartial culture. 
#n is number of candidates, electorate_size is the number of voters.
n=3
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))



column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]


plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

for a in range(50000):
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
    
    if plurality_spoiler ==False and n!=3:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True
                
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
    
    if IRV_spoiler_exists==False and n!=3:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)

    
    

KeyboardInterrupt: 

In [8]:
#IAC simulations
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap

def soc_inac(ranksmaps, rankmapcount, ncand):
    soc_inac_ensemble = []
    condi_position = pd.DataFrame(
        ranksmaps, columns=[i for i in range(1, ncand+1)])
    for i in range(1, ncand+1):
        sum_nsup = 0
        sum_ninf = 0
        nsup = condi_position[condi_position[i] <= int(ncand/2)].index.values
        ninf = condi_position[condi_position[i] > np.ceil(ncand/2)].index.values
        for j in range(len(nsup)):
            sum_nsup = sum_nsup + rankmapcount[nsup[j]]
        for j in range(len(ninf)):
            sum_ninf = sum_ninf + rankmapcount[ninf[j]]
        if sum_ninf > sum_nsup:
            soc_inac_ensemble.append(str(i))

    return soc_inac_ensemble
def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=501
possible_ballots=[]
candmap = gen_cand_map(n)
election=gen_votemap_iac(electorate_size,candmap)

plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    print(a)
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                print("old plurality winner", plurality_winners)
                print("new plurality winner", new_plur_winners)
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                print(data_copy)
                print("plurality spoiler ",l)
                clone_set=clones(data_copy, l)
                print("Clones",clone_set)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)

100
99
98
97
96
95
94
93
old plurality winner ['2']
new plurality winner ['3']
  ballot  Count
0    321     51
1    213    144
2    132    100
3    312    122
4    123      4
5    231     80
plurality spoiler  1


KeyboardInterrupt: 

In [ ]:
#IAC-SP simulations


n=3
electorate_size=501
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))





In [270]:
#Linear-SP simulations


n=4
electorate_size=400
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]
plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

a=100000
count=0
while a>0:
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
                
    
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True
                
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print(count)



Num elections with a plurality spoiler cand:  58906
Num elections with a plurality spoiler set:  90403
Num elections with an IRV spoiler cand:  43959
Num elections with an IRV spoiler set:  57805
132473


In [ ]:
#impartial culture, where we also check for clones
#n is number of candidates, electorate_size is the number of voters.
n=4
electorate_size=400
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))



column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]


plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0

for a in range(100):
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)

print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)

print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)

print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)

print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)



    
    

In [36]:
#Linear-SP simulations, where we also check for clones


n=4
electorate_size=600
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]


plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0

counter=0

a=100000
count=0
while a>0:
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
                        else:
                            if counter<1:
                                print(data)
                                print("spoiler ",l)
                                counter+=1
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)


  ballot  Count
0   0123    201
1   1023    183
2   1203      7
3   1230      0
4   2103      5
5   2130      7
6   2310    183
7   3210     14
spoiler  2
Num elections with a plurality spoiler cand:  58312
Num elections with a plurality spoiler set:  90380
Num elections with an IRV spoiler cand:  43651
Num elections with an IRV spoiler set:  57814
Num elections with a plurality clone spoiler  0
Num elections with a plurality clone spoiler who spoiled the election for their clone 0
Num elections with a plurality almost clone spoiler  12159
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 11339
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  2290
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 1635


In [1]:
#Linear-SP simulations, where we also check for clones


n=3
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]


plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0

counter=0

a=100000
count=0
while a>0:
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                print(data_copy)
                
                clone_set=clones(data_copy, l)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
                        else:
                            if counter<1:
                                print(data)
                                print("spoiler ",l)
                                counter+=1
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)



NameError: name 'all_different' is not defined

In [22]:
#impartial culture. 
#n is number of candidates, electorate_size is the number of voters.
n=3
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))



column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

tie_issue=0
plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

a=20000

while a>0:
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    if max(scores)<=electorate_size/2:
        cands=[str(i) for i in range(n)]
        r=[x for x in powerset(cands)]

        #Check for plurality spoiler candidates and sets

        #First check for spoiler candidates
        plurality_spoiler=False
        plurality_spoiler_set=False

        for l in range(n):
            if str(l) not in plurality_winners:
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                #print(data_copy)
                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    if len(plurality_winners)>1 or len(new_plur_winners)>1:
                        tie_issue+=1
                    plurality_spoiler=True
                    plurality_spoiler_set=True

        if plurality_spoiler==False:
            for subset in r:
                if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                    #print(l)
                    data_copy=data.copy(deep=True)
                    for b in range(len(data_copy)):
                        for l in subset:
                            data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                    new_plur_winners=plurality(data_copy)
                    #print(new_plur_winners)
                    if plurality_winners!=new_plur_winners:
                        plurality_spoiler_set=True


        if plurality_spoiler:
            plur_spoiler+=1
        if plurality_spoiler_set:
            plur_spoiler_set+=1

        IRV_spoiler_exists=False
        IRV_spoiler_set_exists=False
        #Check for IRV spoiler candidates and sets
        #First, check for spoiler cands
        for l in range(n):
            if str(l) not in IRV_winners:
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                if IRV_winners!=new_IRV_winners:
                    IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True

        if IRV_spoiler_exists==False:
            for subset in r:
                if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                    #print(l)
                    data_copy=data.copy(deep=True)
                    for b in range(len(data_copy)):
                        for l in subset:
                            data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                    new_IRV_winners=IRV(data_copy)
                    #print(new_plur_winners)
                    if IRV_winners!=new_IRV_winners:
                        if len(subset)==1:
                            IRV_spoiler_exists=True
                        IRV_spoiler_set_exists=True

        if IRV_spoiler_exists:
            IRV_spoiler+=1
        if IRV_spoiler_set_exists:
            IRV_spoiler_set+=1
        a-=1
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with a plurality spoiler cand and there's a tie issue: ",tie_issue)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)

    
    

Num elections with a plurality spoiler cand:  6296
Num elections with a plurality spoiler set:  6296
Num elections with a plurality spoiler cand and there's a tie issue:  570
Num elections with an IRV spoiler cand:  2467
Num elections with an IRV spoiler set:  2467


In [347]:
#impartial culture, where we also check for clones


n=4
electorate_size=401
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i!=j and k!=i and k!=j:
                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if i!=j and k!=i and k!=j and l!=i and l!=j and l!=k:
                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]


plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0



for a in range(100000):
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners:
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
                
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)



Num elections with a plurality spoiler cand:  49537
Num elections with a plurality spoiler set:  59776
Num elections with an IRV spoiler cand:  22981
Num elections with an IRV spoiler set:  25812
Num elections with a plurality clone spoiler  0
Num elections with a plurality clone spoiler who spoiled the election for their clone 0
Num elections with a plurality almost clone spoiler  0
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 0
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  0
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 0


In [24]:
#IAC simulations
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=3
electorate_size=401
possible_ballots=[]
candmap = gen_cand_map(n)
election=gen_votemap_iac(electorate_size,candmap)

plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l+1): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(1,n+1)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(1,n+1):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(1,n+1):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)

Num elections with a plurality spoiler cand:  40502
Num elections with a plurality spoiler set:  40502
Num elections with an IRV spoiler cand:  21688
Num elections with an IRV spoiler set:  21688
Num elections with a plurality clone spoiler  7
Num elections with a plurality clone spoiler who spoiled the election for their clone 7
Num elections with a plurality almost clone spoiler  1897
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 1842
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  206
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 206


In [26]:
#IAC simulations
pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

n=4
electorate_size=400
possible_ballots=[]
candmap = gen_cand_map(n)
election=gen_votemap_iac(electorate_size,candmap)

plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l+1): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(1,n+1)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(1,n+1):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(1,n+1):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)

Num elections with a plurality spoiler cand:  38004
Num elections with a plurality spoiler set:  48769
Num elections with an IRV spoiler cand:  21612
Num elections with an IRV spoiler set:  24374
Num elections with a plurality clone spoiler  0
Num elections with a plurality clone spoiler who spoiled the election for their clone 0
Num elections with a plurality almost clone spoiler  0
Num elections with a plurality almost clone spoiler who spoiled the election for their clone 0
Num elections with an IRV clone spoiler  0
Num elections with an IRV clone spoiler who spoiled the election for their clone 0
Num elections with an IRV almost clone spoiler  0
Num elections with an IRV almost clone spoiler who spoiled the election for their clone 0


In [4]:
#IAC-SP simulations, where we check for clones
n=3
electorate_size=601
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}
    
    ReplaceSize = 0
    
    ICsize=len(possible_ballots)
    ballot_dict={i:possible_ballots[i] for i in range(len(possible_ballots))}
    for x in range(numvotes):
        
        
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            
            #tvote = gen_ic_vote(alts, rng)
            x=random.randint(0,len(ballot_dict)-1)
            tvote=ballot_dict[x]
            
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    
    return voteMap


def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)




plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=100000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    clone_spoiler_found_plur=False
    clone_spoiler_change_winner_found_plur=False
    almost_clone_spoiler_found_plur=False
    almost_clone_spoiler_change_winner_found_plur=False
    
    for l in range(0,n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            
            if plurality_winners!=new_plur_winners:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                print(data_copy)
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_plur=True
                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            clone_spoiler_change_winner_found_plur=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_plur=True

                    for clone in clone_set:
                        if clone in new_plur_winners and clone not in plurality_winners:
                            almost_clone_spoiler_change_winner_found_plur=True
                
    if plurality_spoiler==False:
        for subset in r:
            if len(subset)>=2 and set(plurality_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')

                new_plur_winners=plurality(data_copy)
                #print(new_plur_winners)
                if plurality_winners!=new_plur_winners:
                    plurality_spoiler_set=True

            
          
    if plurality_spoiler:
        plur_spoiler+=1
    if plurality_spoiler_set:
        plur_spoiler_set+=1
    if clone_spoiler_found_plur:
        plur_clone+=1
    if clone_spoiler_change_winner_found_plur:
        plur_clone_spoiler+=1
    if almost_clone_spoiler_found_plur:
        plur_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_plur:
        plur_almost_clone_spoiler+=1
    
    clone_spoiler_found_IRV=False
    clone_spoiler_change_winner_found_IRV=False
    almost_clone_spoiler_found_IRV=False
    almost_clone_spoiler_change_winner_found_IRV=False
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                
                #Check if l is a clone
                data_copy=data.copy(deep=True)
                
                clone_set=clones(data_copy, l)
                
                almost_clone_set=almost_clones(data_copy, l,p)
                
                if clone_set!=[]:
                    #If l is a clone, see if l spoiled the election for their clone
                    clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            clone_spoiler_change_winner_found_IRV=True
                    
                #Check if l is almost a clone
                data_copy=data.copy(deep=True)
                clone_set=almost_clones(data_copy, l,p)
                if clone_set!=[]:
                    #If l is an almost clone, see if l spoiled the election for their clone
                    almost_clone_spoiler_found_IRV=True
                    for clone in clone_set:
                        if clone in new_IRV_winners and clone not in IRV_winners:
                            almost_clone_spoiler_change_winner_found_IRV=True
    
    if IRV_spoiler_exists==False:
        for subset in r:
            if len(subset)>=2 and set(IRV_winners).isdisjoint(set(subset)):
                #print(l)
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    for l in subset:
                        data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
                new_IRV_winners=IRV(data_copy)
                #print(new_plur_winners)
                if IRV_winners!=new_IRV_winners:
                    if len(subset)==1:
                        IRV_spoiler_exists=True
                    IRV_spoiler_set_exists=True
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    if clone_spoiler_found_IRV:
        IRV_clone+=1
    if clone_spoiler_change_winner_found_IRV:
        IRV_clone_spoiler+=1
    if almost_clone_spoiler_found_IRV:
        IRV_almost_clone+=1
    if almost_clone_spoiler_change_winner_found_IRV:
        IRV_almost_clone_spoiler+=1
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with a plurality spoiler set: " ,plur_spoiler_set)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections with an IRV spoiler set: ", IRV_spoiler_set)
print("Num elections with a plurality clone spoiler ",plur_clone)
print("Num elections with a plurality clone spoiler who spoiled the election for their clone",plur_clone_spoiler)
print("Num elections with a plurality almost clone spoiler ",plur_almost_clone)
print("Num elections with a plurality almost clone spoiler who spoiled the election for their clone",plur_almost_clone_spoiler)
print("Num elections with an IRV clone spoiler ",IRV_clone)
print("Num elections with an IRV clone spoiler who spoiled the election for their clone",IRV_clone_spoiler)
print("Num elections with an IRV almost clone spoiler ",IRV_almost_clone)
print("Num elections with an IRV almost clone spoiler who spoiled the election for their clone",IRV_almost_clone_spoiler)
print("Model=",model," n=",n," V=",electorate_size) 


  ballot  Count
0    210     75
1    102    234
2    012    290
3    120      2
  ballot  Count
0    120    178
1    102     31
2    210    237
3    012    155
  ballot  Count
0    012    285
1    210    124
2    102    129
3    120     63
  ballot  Count
0    102    138
1    012    118
2    210    259
3    120     86
  ballot  Count
0    012    209
1    210    254
2    120     10
3    102    128
  ballot  Count
0    012    209
1    210    254
2    120     10
3    102    128
  ballot  Count
0    120    133
1    102    102
2    012    131
3    210    235
  ballot  Count
0    120    201
1    012    298
2    102     82
3    210     20
  ballot  Count
0    210    261
1    012    260
2    120     65
3    102     15
  ballot  Count
0    012    253
1    120     68
2    210    263
3    102     17
  ballot  Count
0    012    117
1    210    295
2    102    189
  ballot  Count
0    012    117
1    210    295
2    102    189
  ballot  Count
0    012    277
1    210     94
2    102    210
3    120

KeyboardInterrupt: 

In [25]:


#Linear-SP simulations for CL


n=5
electorate_size=400
possible_ballots=[]
model='Linear-SP'


if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=100000
count=0
while a>0:
    
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
    
        CL=Cond_loser(data)

        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            #print("CL is a plurality winner ",CL)
            #print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                #print("CL is a plurality spoiler ",CL)
                #print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            #print("CL is an IRV winner ",CL)
            #print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                #print("CL is an IRV spoiler ",CL)
                #print(data)
                CL_IRV_spoiler+=1
        a-=1
print("model=", model)
print("n= ",n)
print("V= ",electorate_size)          
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)



model= Linear-SP
n=  5
V=  400
Number of elections in which CL is a plurality spoiler  29980
Number of elections in which CL is a plurality winner  8518
Number of elections in which CL is an IRV spoiler  17436
Number of elections in which CL is an IRV winner  230


In [33]:

#IAC-SP for CL
n=5
electorate_size=600
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}
    
    ReplaceSize = 0
    
    ICsize=len(possible_ballots)
    ballot_dict={i:possible_ballots[i] for i in range(len(possible_ballots))}
    for x in range(numvotes):
        
        
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            
            #tvote = gen_ic_vote(alts, rng)
            x=random.randint(0,len(ballot_dict)-1)
            tvote=ballot_dict[x]
            
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    
    return voteMap


def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=100000
count=0
while a>0:
    
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
    
        CL=Cond_loser(data)

        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            #print("CL is a plurality winner ",CL)
            #print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                #print("CL is a plurality spoiler ",CL)
                #print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            #print("CL is an IRV winner ",CL)
            #print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                #print("CL is an IRV spoiler ",CL)
                #print(data)
                CL_IRV_spoiler+=1
        a-=1
print("model=", model)
print("n= ",n)
print("V= ",electorate_size)          
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)




model= IAC-SP
n=  5
V=  600
Number of elections in which CL is a plurality spoiler  9821
Number of elections in which CL is a plurality winner  79
Number of elections in which CL is an IRV spoiler  815
Number of elections in which CL is an IRV winner  3


In [ ]:


#IAC for CL
n=3
electorate_size=601
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}

    ICsize = math.factorial(len(alts))
    ReplaceSize = 0

    for x in range(numvotes):
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            tvote = gen_ic_vote(alts, rng)
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    return voteMap

def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap

def soc_inac(ranksmaps, rankmapcount, ncand):
    soc_inac_ensemble = []
    condi_position = pd.DataFrame(
        ranksmaps, columns=[i for i in range(1, ncand+1)])
    for i in range(1, ncand+1):
        sum_nsup = 0
        sum_ninf = 0
        nsup = condi_position[condi_position[i] <= int(ncand/2)].index.values
        ninf = condi_position[condi_position[i] > np.ceil(ncand/2)].index.values
        for j in range(len(nsup)):
            sum_nsup = sum_nsup + rankmapcount[nsup[j]]
        for j in range(len(ninf)):
            sum_ninf = sum_ninf + rankmapcount[ninf[j]]
        if sum_ninf > sum_nsup:
            soc_inac_ensemble.append(str(i))

    return soc_inac_ensemble
def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)

column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=10
count=0
while a>0:
    
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l+1): 
                scores[l]+=data.iloc[k]['Count']
    print("\n")
    print(data)
    print("scores",scores)
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
        print("inside if")
        CL=Cond_loser(data)
        print("CL is ",CL)
        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            print("CL is a plurality winner ",CL)
            print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                print("CL is a plurality spoiler ",CL)
                print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            print("CL is an IRV winner ",CL)
            print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                print("CL is an IRV spoiler ",CL)
                print(data)
                CL_IRV_spoiler+=1
        a-=1
print("model=", model)
print("n= ",n)
print("V= ",electorate_size)          
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)





In [10]:


#impartial culture for CL

model='IC'
#n=5
electorate_size=601

n_values=[3,4,5]     
for n in n_values:
    possible_ballots=[]

    if n==3:
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    if all_different([i,j,k]):
                        
                        possible_ballots.append(str(i)+str(j)+str(k))
    if n==4:
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    for l in range(n):
                        if all_different([i,j,k,l]):
                            
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

    if n==5:
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    for l in range(n):
                        for p in range(n):
                            if all_different([i,j,k,l,p]):
                                
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

    if n==6:
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    for l in range(n):
                        for p in range(n):
                            for q in range(n):
                                if all_different([i,j,k,l,p,q]):
                                    
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))


    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]

    CL_plurality_spoiler=0
    CL_plurality_winner=0
    CL_IRV_spoiler=0
    CL_IRV_winner=0

    a=100000
    count=0
    while a>0:

        count+=1
        data['ballot']=possible_ballots
        data['Count']=[0 for item in possible_ballots]
        for k in range(electorate_size):
            vote=random.randint(0,len(possible_ballots)-1)
            data.at[vote,'Count']+=1
        #print(data)
        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        #print(IRV_winners)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)
        #print(plurality_winners)
        #print("CL",type(Cond_loser(data)))
        #Check for majority candidate and CL
        scores=[0 for i in range(n)]
        for k in range(len(data)):
            for l in range(n):
                if data.iloc[k]['ballot'][0]==str(l): 
                    scores[l]+=data.iloc[k]['Count']



        if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:

            CL=Cond_loser(data)

            data_copy=data.copy(deep=True)    
            IRV_winners=IRV(data_copy)
            data_copy=data.copy(deep=True)  
            plurality_winners=plurality(data_copy)

            #First check for spoiler candidates


            if CL in plurality_winners:
                #print("CL is a plurality winner ",CL)
                #print(data)
                CL_plurality_winner+=1

            if CL not in plurality_winners:
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
                #print(data_copy)
                new_plur_winners=plurality(data_copy)
                if plurality_winners!=new_plur_winners:
                    #print("CL is a plurality spoiler ",CL)
                    #print(data)
                    CL_plurality_spoiler+=1
            if CL in IRV_winners:
                CL_IRV_winner+=1
                #print("CL is an IRV winner ",CL)
                #print(data)

            if CL not in IRV_winners:
                data_copy=data.copy(deep=True)
                for b in range(len(data_copy)):
                    data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
                #print(data_copy)
                new_IRV_winners=IRV(data_copy)
                if IRV_winners!=new_IRV_winners:
                    #print("CL is an IRV spoiler ",CL)
                    #print(data)
                    CL_IRV_spoiler+=1
            a-=1
            if a%10000==0:
                print(a)
    print("model=", model)
    print("n= ",n)
    print("V= ",electorate_size)          
    print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
    print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
    print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
    print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)




90000
80000
70000
60000
50000
40000
30000
20000
10000
0
model= IC
n=  3
V=  601
Number of elections in which CL is a plurality spoiler  20833
Number of elections in which CL is a plurality winner  4054
Number of elections in which CL is an IRV spoiler  3752
Number of elections in which CL is an IRV winner  238
90000
80000
70000
60000
50000
40000
30000
20000
10000
0
model= IC
n=  4
V=  601
Number of elections in which CL is a plurality spoiler  26677
Number of elections in which CL is a plurality winner  2722
Number of elections in which CL is an IRV spoiler  4388
Number of elections in which CL is an IRV winner  125
90000
80000
70000
60000
50000
40000
30000
20000
10000
0
model= IC
n=  5
V=  601
Number of elections in which CL is a plurality spoiler  28383
Number of elections in which CL is a plurality winner  2191
Number of elections in which CL is an IRV spoiler  4575
Number of elections in which CL is an IRV winner  57


In [17]:
File=open('Desktop/ward1.xlsx','r')

In [18]:
File

<_io.TextIOWrapper name='Desktop/ward1.xlsx' mode='r' encoding='UTF-8'>

In [16]:
lines=File.readlines()
lines

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte

In [16]:
#Linear-SP simulations


n=3
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]
plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

a=100000
count=0
A_wins=0
A_wins_IRV=0
A_wins_C_spoils=0
A_wins_B_spoils=0
A_wins_C_spoils_IRV=0
A_wins_B_spoils_IRV=0
while a>0:
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    if plurality_winners==['0']:
        A_wins+=1
    if IRV_winners==['0']:
        A_wins_IRV+=1
    
    
    cands=[str(i) for i in range(n)]
    
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners and plurality_winners==['0']:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                if str(l)=='1':
                    A_wins_B_spoils+=1
                if str(l)=='2':
                    A_wins_C_spoils+=1
                
                
    if plurality_spoiler:
        plur_spoiler+=1
    
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners and IRV_winners==['0']:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                if str(l)=='1':
                    A_wins_B_spoils_IRV+=1
                if str(l)=='2':
                    A_wins_C_spoils_IRV+=1
    
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections A wins plurality ",A_wins)
print("Num elections A wins IRV ",A_wins_IRV)
print("Num elections A wins pluarlity, B spiols, ",A_wins_B_spoils)
print("Num elections A wins pluarlity, C spiols, ",A_wins_C_spoils)
print("Num elections A wins IRV, B spiols, ",A_wins_B_spoils_IRV)
print("Num elections A wins IRV, C spiols, ",A_wins_C_spoils_IRV)
print(count)




Num elections with a plurality spoiler cand:  38692
Num elections with an IRV spoiler cand:  19462
Num elections A wins plurality  38692
Num elections A wins IRV  19462
Num elections A wins pluarlity, B spiols,  10903
Num elections A wins pluarlity, C spiols,  38692
Num elections A wins IRV, B spiols,  52
Num elections A wins IRV, C spiols,  19462
199384


In [18]:
#Linear-SP simulations, figuring out who wins and who spoils


n=4
electorate_size=601
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]
plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

a=100000
count=0
A_wins=0
A_wins_IRV=0
A_wins_B_spoils=0
A_wins_C_spoils=0
A_wins_D_spoils=0
A_wins_B_spoils_IRV=0
A_wins_C_spoils_IRV=0
A_wins_D_spoils_IRV=0

0
A_wins_D_spoils_IRV=0
while a>0:
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    if plurality_winners==['0']:
        A_wins+=1
    if IRV_winners==['0']:
        A_wins_IRV+=1
    
    
    cands=[str(i) for i in range(n)]
    
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    plurality_spoiler=False
    plurality_spoiler_set=False
    
    for l in range(n):
        if str(l) not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            #print(new_plur_winners)
            if plurality_winners!=new_plur_winners and plurality_winners==['0']:
                
                plurality_spoiler=True
                plurality_spoiler_set=True
                if str(l)=='1':
                    A_wins_B_spoils+=1
                if str(l)=='2':
                    A_wins_C_spoils+=1
                if str(l)=='3':
                    A_wins_D_spoils+=1
                
                
                
    if plurality_spoiler:
        plur_spoiler+=1
    
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    #Check for IRV spoiler candidates and sets
    #First, check for spoiler cands
    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners and IRV_winners==['0']:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                if str(l)=='1':
                    A_wins_B_spoils_IRV+=1
                if str(l)=='2':
                    A_wins_C_spoils_IRV+=1
                if str(l)=='3':
                    A_wins_D_spoils_IRV+=1
                
    
                
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    
    a-=1
                
print("Num elections with a plurality spoiler cand: " ,plur_spoiler)
print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections A wins plurality ",A_wins)
print("Num elections A wins IRV ",A_wins_IRV)
print("Num elections A wins pluarlity, B spiols, ",A_wins_B_spoils)
print("Num elections A wins pluarlity, C spiols, ",A_wins_C_spoils)
print("Num elections A wins pluarlity, D spiols, ",A_wins_D_spoils)
print("Num elections A wins IRV, B spiols, ",A_wins_B_spoils_IRV)
print("Num elections A wins IRV, C spiols, ",A_wins_C_spoils_IRV)
print("Num elections A wins IRV, D spiols, ",A_wins_D_spoils_IRV)

print(count)





Num elections with a plurality spoiler cand:  19235
Num elections with an IRV spoiler cand:  11068
Num elections A wins plurality  34940
Num elections A wins IRV  18174
Num elections A wins pluarlity, B spiols,  2676
Num elections A wins pluarlity, C spiols,  12311
Num elections A wins pluarlity, D spiols,  17268
Num elections A wins IRV, B spiols,  811
Num elections A wins IRV, C spiols,  3878
Num elections A wins IRV, D spiols,  7838
132672


In [ ]:


#Linear-SP simulations for CL


n=3
electorate_size=601
possible_ballots=[]
model='Linear-SP'


if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))
                
if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))
                                
if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))
column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0
CL_IRV_winner=0

a=100000
count=0
while a>0:
    
    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    cand_positions=[]
    
    for k in range(n):
        pos=random.randint(0,electorate_size-1)
        while pos in cand_positions:
            pos=random.randint(0,electorate_size-1)
        cand_positions.append(pos)

    cand_positions.sort()
    cand_dict={cand_positions[i]:str(i) for i in range(n)}
    
    
    #Get each voter's choice and add to the dataframe
    for k in range(electorate_size):
        choice=''
        
        dict1=cand_dict.copy()
        
        while len(choice)<n:
            distances=[abs(k-i) for i in dict1]
            distances.sort()
            if distances[0]==0:
                choice+=str(cand_dict[k])
    
            else:
                if k-distances[0] in cand_positions and k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k-distances[0])
                    dict1.pop(k+distances[0])
                elif k-distances[0] in cand_positions:
                    choice+=str(cand_dict[k-distances[0]])
                    dict1.pop(k-distances[0])
                elif k+distances[0] in cand_positions:
                    choice+=str(cand_dict[k+distances[0]])
                    dict1.pop(k+distances[0])
        
        for l in range(len(data)):
            if data.at[l,'ballot']==choice:
                index=l
                break
        data.at[index,'Count']+=1
    
    
    #Check for majority candidate and CL
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)<=electorate_size/2 and Cond_loser(data)!=False:
    
        CL=Cond_loser(data)
        

        data_copy=data.copy(deep=True)    
        IRV_winners=IRV(data_copy)
        data_copy=data.copy(deep=True)  
        plurality_winners=plurality(data_copy)

        #First check for spoiler candidates


        if CL in plurality_winners:
            #print("CL is a plurality winner ",CL)
            #print(data)
            CL_plurality_winner+=1

        if CL not in plurality_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_plur_winners=plurality(data_copy)
            if plurality_winners!=new_plur_winners:
                #print("CL is a plurality spoiler ",CL)
                #print(data)
                CL_plurality_spoiler+=1
        if CL in IRV_winners:
            CL_IRV_winner+=1
            #print("CL is an IRV winner ",CL)
            #print(data)

        if CL not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(CL,'')
            #print(data_copy)
            new_IRV_winners=IRV(data_copy)
            if IRV_winners!=new_IRV_winners:
                #print("CL is an IRV spoiler ",CL)
                #print(data)
                CL_IRV_spoiler+=1
        a-=1
print("model=", model)
print("n= ",n)
print("V= ",electorate_size)          
print("Number of elections in which CL is a plurality spoiler ", CL_plurality_spoiler)
print("Number of elections in which CL is a plurality winner ", CL_plurality_winner)
print("Number of elections in which CL is an IRV spoiler ", CL_IRV_spoiler)
print("Number of elections in which CL is an IRV winner ", CL_IRV_winner)




In [17]:


#impartial culture for CL, just messing around to see why graphs are the way they are

model='IC'
#n=5
electorate_size=601

n=3
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):

                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):

                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):

                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):

                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))


column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0

num_times_A_CL=0
num_times_B_CL=0
num_times_C_CL=0
num_votes_for_A=0
A_scores=[]
a=100000
count=0
while a>0:

    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    #print(data)
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    #print(IRV_winners)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    #print(plurality_winners)
    #print("CL",type(Cond_loser(data)))
    

    

    if Cond_loser(data)!=False:
        CL=Cond_loser(data)
        if CL=='0':
            num_times_A_CL+=1
            A_score=0
            for k in range(len(data)):
                
                if data.iloc[k]['ballot'][0]=='0':
                    A_score+=data.iloc[k]['Count']
            num_votes_for_A+=A_score
            A_scores.append(A_score)
        
        
        a-=1
        
        
print(num_times_A_CL)
print(num_votes_for_A/num_times_A_CL)



33566
189.66376690698922


In [18]:
import statistics
print(min(A_scores))
print(max(A_scores))
print(statistics.pstdev(A_scores))

142
220
8.18535277187245


In [19]:


#impartial culture for CL, just messing around to see why graphs are the way they are

model='IC'
#n=5
electorate_size=601

n=5
possible_ballots=[]

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):

                    possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):

                        possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):

                            possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):

                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))


column_names=['ballot','Count']
data=pd.DataFrame(columns = column_names)
data['ballot']=possible_ballots
data['Count']=[0 for item in possible_ballots]

CL_plurality_spoiler=0
CL_plurality_winner=0
CL_IRV_spoiler=0

num_times_A_CL=0
num_votes_for_A=0
A_scores=[]

a=100000
count=0
while a>0:

    count+=1
    data['ballot']=possible_ballots
    data['Count']=[0 for item in possible_ballots]
    for k in range(electorate_size):
        vote=random.randint(0,len(possible_ballots)-1)
        data.at[vote,'Count']+=1
    #print(data)
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    #print(IRV_winners)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    #print(plurality_winners)
    #print("CL",type(Cond_loser(data)))
    #Check for majority candidate and CL
    

    

    if Cond_loser(data)!=False:
        CL=Cond_loser(data)
        if CL=='0':
            num_times_A_CL+=1
            A_score=0
            for k in range(len(data)):
                
                if data.iloc[k]['ballot'][0]=='0':
                    A_score+=data.iloc[k]['Count']
            num_votes_for_A+=A_score
            A_scores.append(A_score)
        
        a-=1
        
        
print(num_times_A_CL)
print(num_votes_for_A/num_times_A_CL)




20107
110.65509524046352


In [55]:
#IAC-SP simulations, where we check for clones
n=5
electorate_size=601
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}
    
    ReplaceSize = 0
    
    ICsize=len(possible_ballots)
    ballot_dict={i:possible_ballots[i] for i in range(len(possible_ballots))}
    for x in range(numvotes):
        
        
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            
            #tvote = gen_ic_vote(alts, rng)
            x=random.randint(0,len(ballot_dict)-1)
            tvote=ballot_dict[x]
            
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    
    return voteMap


def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)




plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
   
    
C_wins_and_spoiler_exists=0
D_wins_and_B_spoiler=0
D_wins_and_C_spoiler=0
D_wins_and_no_spoiler=0
a=20000
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    
    
    
    cands=[str(i) for i in range(n)]
    r=[x for x in powerset(cands)]
    
    #Check for plurality spoiler candidates and sets
    
    #First check for spoiler candidates
    IRV_spoiler_exists=False
    IRV_spoiler_set_exists=False
    

    for l in range(n):
        if str(l) not in IRV_winners:
            data_copy=data.copy(deep=True)
            for b in range(len(data_copy)):
                data_copy.at[b,'ballot']=data_copy.at[b,'ballot'].replace(str(l),'')
            new_IRV_winners=IRV(data_copy)
            
            if IRV_winners!=new_IRV_winners:
                IRV_spoiler_exists=True
                IRV_spoiler_set_exists=True
                if IRV_winners==['2']:
                    C_wins_and_spoiler_exists+=1
                    print("C wins and there's a spoiler ", l)
                    print(data)
                if IRV_winners==['3'] and str(l)=='1':
                    D_wins_and_B_spoiler+=1
                    #print("D wins and B is a spoiler")
                    #print(data)
                if IRV_winners==['3'] and str(l)=='2':
                    D_wins_and_C_spoiler+=1
    if IRV_spoiler_exists==False and IRV_winners==['3']:
        D_wins_and_no_spoiler+=1       
    if IRV_spoiler_exists:
        IRV_spoiler+=1
    if IRV_spoiler_set_exists:
        IRV_spoiler_set+=1
    
    a-=1
                

print("Num elections with an IRV spoiler cand: ", IRV_spoiler)
print("Num elections where C wins and a spoiler exists ",C_wins_and_spoiler_exists)
print("Num elections where D wins and B is spoiler ",D_wins_and_B_spoiler)
print("Num elections where D wins and C is spoiler ",D_wins_and_C_spoiler)
print("Num elections where D wins and no spoiler ",D_wins_and_no_spoiler)



 



C wins and there's a spoiler  4
   ballot  Count
0   32140     18
1   21304     34
2   23140     49
3   43210    130
4   12340     54
5   34210     24
6   10234     11
7   23104     33
8   32410     61
9   01234     68
10  12304     40
11  21340     22
12  32104     10
13  12034     12
14  21034     29
15  23410      6
C wins and there's a spoiler  0
   ballot  Count
0   21304     31
1   32104     42
2   43210     91
3   12034     20
4   12304     55
5   01234     92
6   34210     29
7   32140     64
8   21340     67
9   23410     16
10  23140     19
11  32410     17
12  23104     15
13  12340     14
14  21034     29
C wins and there's a spoiler  0
   ballot  Count
0   43210    170
1   01234     96
2   21304      7
3   21034     29
4   12340     31
5   23104     12
6   23140     30
7   32104     56
8   21340     29
9   32140     44
10  10234     12
11  32410     18
12  12034     28
13  34210     21
14  23410     11
15  12304      7
C wins and there's a spoiler  4
   ballot  Count
0   4

KeyboardInterrupt: 

In [45]:
#IAC-SP simulations, where we check for clones
n=3
electorate_size=601
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}
    
    ReplaceSize = 0
    
    ICsize=len(possible_ballots)
    ballot_dict={i:possible_ballots[i] for i in range(len(possible_ballots))}
    for x in range(numvotes):
        
        
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            
            #tvote = gen_ic_vote(alts, rng)
            x=random.randint(0,len(ballot_dict)-1)
            tvote=ballot_dict[x]
            
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    
    return voteMap


def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)




plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=20000
A_wins_IRV=0
B_wins_IRV=0
C_wins_IRV=0

A_wins_plurality=0
B_wins_plurality=0
C_wins_plurality=0
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    if IRV_winners==['0']:
        A_wins_IRV+=1
    if IRV_winners==['1']:
        B_wins_IRV+=1
    if IRV_winners==['2']:
        C_wins_IRV+=1
    if plurality_winners==['0']:
        A_wins_plurality+=1
    if plurality_winners==['1']:
        B_wins_plurality+=1
    if plurality_winners==['2']:
        C_wins_plurality+=1
    
    a-=1
                
print(A_wins_IRV)
print(B_wins_IRV)
print(C_wins_IRV)

print(A_wins_plurality)
print(B_wins_plurality)
print(C_wins_plurality)




1872
16090
2038
5417
8837
5625


In [46]:
#IAC-SP simulations, where we check for clones
n=5
electorate_size=601
possible_ballots=[]
model='IAC-SP'
    

if n==3:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if all_different([i,j,k]):
                    if is_SP_ranking([i,j,k]):
                        possible_ballots.append(str(i)+str(j)+str(k))
if n==4:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    if all_different([i,j,k,l]):
                        if is_SP_ranking([i,j,k,l]):
                            possible_ballots.append(str(i)+str(j)+str(k)+str(l))

if n==5:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        if all_different([i,j,k,l,p]):
                            if is_SP_ranking([i,j,k,l,p]):
                                possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p))

if n==6:
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    for p in range(n):
                        for q in range(n):
                            if all_different([i,j,k,l,p,q]):
                                if is_SP_ranking([i,j,k,l,p,q]):
                                    possible_ballots.append(str(i)+str(j)+str(k)+str(l)+str(p)+str(q))




pd.options.mode.chained_assignment = None  # default='warn'
def gen_ic_vote(alts, seed=None):
    rng = np.random.default_rng(seed)
    alts=[str(alt) for alt in alts]
    options = list(alts)
    rng.shuffle(options)
    return tuple(options)

def gen_urn(numvotes, replace, alts):
    rng = np.random.default_rng()
    voteMap = {}
    ReplaceVotes = {}
    
    ReplaceSize = 0
    
    ICsize=len(possible_ballots)
    ballot_dict={i:possible_ballots[i] for i in range(len(possible_ballots))}
    for x in range(numvotes):
        
        
        # flip = random.randint(1, ICsize+ReplaceSize)
        flip = rng.integers(1, ICsize+ReplaceSize, endpoint=True)
        
        if flip <= ICsize:
            #generate an IC vote and make a suitable number of replacements...
            # tvote = gen_ic_vote(alts)
            
            #tvote = gen_ic_vote(alts, rng)
            x=random.randint(0,len(ballot_dict)-1)
            tvote=ballot_dict[x]
            
            voteMap[tvote] = (voteMap.get(tvote, 0) + 1)
            ReplaceVotes[tvote] = (ReplaceVotes.get(tvote, 0) + replace)
            ReplaceSize += replace
        else:
            #iterate over replacement hash and select proper vote.
            flip = flip - ICsize
            for vote in ReplaceVotes.keys():
                flip = flip - ReplaceVotes[vote]
                if flip <= 0:
                    voteMap[vote] = (voteMap.get(vote, 0) + 1)
                    ReplaceVotes[vote] = (ReplaceVotes.get(vote, 0) + replace)
                    ReplaceSize += replace
                    break
            else:
                print("We Have a problem... replace fell through....")
                exit()
                # return None
    
    return voteMap


def voteset_to_rankmap(votemap, candmap):
    rmapcount = []
    rmap = []
    #Votemaps are tuple --> count, so it's strict and easy...
    for order in votemap.keys():
        rmapcount.append(votemap[order])
        cmap = {}
        for crank in range(1, len(order)+1):
            cmap[order[crank-1]] = crank
        rmap.append(cmap)
    return rmap, rmapcount

def gen_cand_map(nalts):
    candmap = {i:"Candidate " + str(i) for i in range(1, nalts+1)}
    return candmap


def gen_votemap_iac(nvoters, candmap):
    return gen_urn(nvoters, 1, candmap)




plur_spoiler=0
IRV_spoiler=0
plur_spoiler_set=0
IRV_spoiler_set=0

p=0.95
plur_clone=0
plur_clone_spoiler=0
plur_almost_clone=0
plur_almost_clone_spoiler=0

IRV_clone=0
IRV_clone_spoiler=0
IRV_almost_clone=0
IRV_almost_clone_spoiler=0
    
a=20000
A_wins_IRV=0
B_wins_IRV=0
C_wins_IRV=0
D_wins_IRV=0
E_wins_IRV=0

A_wins_plurality=0
B_wins_plurality=0
C_wins_plurality=0
D_wins_plurality=0
E_wins_plurality=0
count=0
while a>0:
    candmap = gen_cand_map(n)
    election=gen_votemap_iac(electorate_size,candmap)
    keys=list(election.keys())
    values=list(election.values())
    values_string=[''.join(key) for key in keys]
    column_names=['ballot','Count']
    data=pd.DataFrame(columns = column_names)
    data['ballot']=values_string
    data['Count']=values
    
    
    #Check for majority candidate
    scores=[0 for i in range(n)]
    for k in range(len(data)):
        for l in range(0,n):
            if data.iloc[k]['ballot'][0]==str(l): 
                scores[l]+=data.iloc[k]['Count']
    
    
    
    if max(scores)>electorate_size/2:
        continue
    
    
    data_copy=data.copy(deep=True)    
    IRV_winners=IRV(data_copy)
    data_copy=data.copy(deep=True)  
    plurality_winners=plurality(data_copy)
    if IRV_winners==['0']:
        A_wins_IRV+=1
    if IRV_winners==['1']:
        B_wins_IRV+=1
    if IRV_winners==['2']:
        C_wins_IRV+=1
    if IRV_winners==['3']:
        D_wins_IRV+=1
    if IRV_winners==['4']:
        E_wins_IRV+=1
    if plurality_winners==['0']:
        A_wins_plurality+=1
    if plurality_winners==['1']:
        B_wins_plurality+=1
    if plurality_winners==['2']:
        C_wins_plurality+=1
    if plurality_winners==['3']:
        D_wins_plurality+=1
    if plurality_winners==['4']:
        E_wins_plurality+=1
    
    a-=1
                
print(A_wins_IRV)
print(B_wins_IRV)
print(C_wins_IRV)
print(D_wins_IRV)
print(E_wins_IRV)
print(A_wins_plurality)
print(B_wins_plurality)
print(C_wins_plurality)
print(D_wins_plurality)
print(E_wins_plurality)





7
2954
14149
2883
7
131
4187
11161
4310
115
